# Úkol: Zpracování Zoom reportů
Vytvořte Python skript, který zpracuje reporty o připojení účastníků on-line kurzu [Praha Coding](https://www.prahacoding.cz) a vygeneruje individuální report /`csv`/ pro každého z účastníků.   
Vstupní data ze tří různých kurzů jsou v samostatných složkách /označených číslem kurzu/ v souboru `data/source/reporty`.   
Výstupní soubory budou ve složce `data/output/reporty/<id-kurzu>` se jménem účastníka /bez diakritiky a mezer/

## Struktura vstupních dat

Jednotlivé lekce jsou v samostatných souborech s údaji o době připojení každého z účastníků:

| Meeting ID,Topic | Start Time | End Time | User Email | Duration (Minutes) | Participants |
| -------------- | ------------------ | ------- | ---------- | -------- | ------- | 
| 95645604158  |  Datova Analytika  |  2024-03-21 18:18:42  |  2024-03-21 21:45:54  |  info@prahacoding.cz  |  207  |  15  |  

| Name (Original Name)  |  User Email  |  Join Time  |  Leave Time  |  Duration (Minutes)  |  Guest  |  Recording Consent  |  In Waiting Room |
| --------------|------------------|-------|----------|--------|-------|-------|-----|
| Zdeňka Heczková  |    |  2024-03-21 18:18:42  |  2024-03-21 21:45:21  |  207  |  Yes  |  Yes  |  No |
| František Matějíček  |    |  2024-03-21 18:18:54  |  2024-03-21 21:41:39  |  203  |  Yes  |  Yes  |  No |
| Ján Rožďalovský  |    |  2024-03-21 18:19:07  |  2024-03-21 21:45:53  |  207  |  Yes  |  Yes  |  No |
| Ludmila Kozáková  |    |  2024-03-21 18:21:39  |  2024-03-21 21:12:26  |  171  |  Yes  |  Yes  |  No |
| Iveta Poloková  |    |  2024-03-21 18:22:03  |  2024-03-21 21:12:22  |  171  |  Yes  |  Yes  |  No |
| Petra Strnadová  |    |  2024-03-21 18:27:10  |  2024-03-21 21:12:23  |  166  |  Yes  |  Yes  |  No |
| Karel Pavlacký  |    |  2024-03-21 18:29:27  |  2024-03-21 21:45:54  |  197  |  Yes  |  Yes  |  No |
| Hana Füllová  |    |  2024-03-21 18:29:32  |  2024-03-21 18:31:16  |  2  |  Yes  |  Yes  |  No |
| Ondřej Koutný  |    |  2024-03-21 18:29:37  |  2024-03-21 21:12:21  |  163  |  Yes  |  Yes  |  No |
| Oskar Koriťák  |    |  2024-03-21 18:29:52  |  2024-03-21 21:12:23  |  163  |  Yes  |  Yes  |  No |
| Hana Füllová  |    |  2024-03-21 18:30:22  |  2024-03-21 18:31:10  |  1  |  Yes  |    |  No |
| Hana Füllová  |    |  2024-03-21 18:31:27  |  2024-03-21 19:02:13  |  31  |  Yes  |    |  No |
| Jiří Boháč  |    |  2024-03-21 18:31:50  |  2024-03-21 21:23:23  |  172  |  Yes  |  Yes  |  No |
| Hana Füllová  |    |  2024-03-21 18:32:08  |  2024-03-21 18:59:35  |  28  |  Yes  |  Yes  |  No |
| Hana Füllová  |    |  2024-03-21 19:02:14  |  2024-03-21 21:12:24  |  131  |  Yes  |    |  No |


## 0. Počáteční Python nezbytnosti

## Importy

In [31]:
import pandas
import glob
import os


## Konstanty a funkce

In [32]:
__file__ = __vsc_ipynb_file__
SCRIPT_FOLDER = os.path.dirname(__file__)
PROJECT_FOLDER = os.path.dirname(SCRIPT_FOLDER)
SOURCE_FOLDER = os.path.join(PROJECT_FOLDER,'data','source','reporty','PCS001')
''' slozka se zdrojovymi Zoom reporty '''
DESTINATION_FOLDER = os.path.join(PROJECT_FOLDER,'data','output','reporty','PCS001')
''' slozka s vyslednymi individualnimi reporty '''
ZOOM_REPORTS = "*.csv"
''' maska nazvu zdrojovych souboru '''

def to_ascii(text):
    ''' replace diacritics in text with ASCII
    '''
    chars_from = 'ÀÁÂÃÄÅÇÈÉÊËÌÍÎÏÑÒÓÔÕÖÙÚÛÜÝßàáâãäåçèéêëìíîïñòóôõöùúûüýÿĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġĢģĤĥĦħĨĩĪīĬĭĮįİıĶķĸĹĺĻļĽľĿŀŁłŃńŅņŇňŉŊŋŌōŎŏŐőŔŕŖŗŘřŚśŜŝŞşŠšŢţŤťŦŧŨũŪūŬŭŮůŰűŲųŴŵŶŷŸŹźŻżŽžſ'
    chars_to  =  'AAAAAACEEEEIIIINOOOOOUUUUYsaaaaaaceeeeiiiinooooouuuuyyAaAaAaCcCcCcCcDdDdEeEeEeEeEeGgGgGgGgHhHhIiIiIiIiIiKkkLlLlLlLlLlNnNnNnNnNOoOoOoRrRrRrSsSsSsSsTtTtTtUuUuUuUuUuUuWwYyYZzZzZzs'
    output = ""
    for chr in text:
        pos = chars_from.find(chr)
        if pos < 0:
            output += chr
        else:
            output += chars_to[pos]
    return output


### Sloupce, které nás zajímají

In [33]:
# z originálu
ZOOM_NAME = 'Name (Original Name)'
ZOOM_JOIN = 'Join Time'
ZOOM_LEAVE = 'Leave Time'

# do výsledného reportu
REPORT_NAME = "Student"
REPORT_JOIN = "Čas připojení"
REPORT_LEAVE = "Čas odpojení"
REPORT_DURATION = "Doba on-line"

# pomocné sloupce
COL_LESSON = 'Lekce'
COL_DURATION = 'Duration'


## 1. Přečti zdrojové soubory a spoj vše do jednoho `DataFrame`

In [34]:
reports_list = []
for file in glob.glob(os.path.join(SOURCE_FOLDER, ZOOM_REPORTS)):
    #print(file)
    df = pandas.read_csv(file,skiprows=2,parse_dates = [ZOOM_JOIN, ZOOM_LEAVE])
    reports_list.append(df)

reports = pandas.concat(reports_list)
reports.info()


<class 'pandas.core.frame.DataFrame'>
Index: 280 entries, 0 to 6
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name (Original Name)  280 non-null    object        
 1   User Email            0 non-null      float64       
 2   Join Time             280 non-null    datetime64[ns]
 3   Leave Time            280 non-null    datetime64[ns]
 4   Duration (Minutes)    280 non-null    int64         
 5   Guest                 280 non-null    object        
 6   Recording Consent     71 non-null     object        
 7   In Waiting Room       280 non-null    object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 19.7+ KB


## 2. Krok za krokem vytvoř individuální reporty

### a. přidej sloupec se dnem konání lekce

In [35]:
reports[COL_LESSON] = reports[ZOOM_JOIN].dt.date
reports

,Name (Original Name),User Email,Join Time,Leave Time,Duration (Minutes),Guest,Recording Consent,In Waiting Room,Lekce
0,Zdeňka Heczková,NaN,2024-03-21 18:18:42,2024-03-21 21:45:21,207,Yes,Yes,No,2024-03-21
1,František Matějíček,NaN,2024-03-21 18:18:54,2024-03-21 21:41:39,203,Yes,Yes,No,2024-03-21
2,Ján Rožďalovský,NaN,2024-03-21 18:19:07,2024-03-21 21:45:53,207,Yes,Yes,No,2024-03-21
3,Ludmila Kozáková,NaN,2024-03-21 18:21:39,2024-03-21 21:12:26,171,Yes,Yes,No,2024-03-21
4,Iveta Poloková,NaN,2024-03-21 18:22:03,2024-03-21 21:12:22,171,Yes,Yes,No,2024-03-21
...,...,...,...,...,...,...,...,...,...
2,Petra Strnadová,NaN,2024-05-07 18:27:05,2024-05-07 19:45:33,79,Yes,NaN,No,2024-05-07
3,Ondřej Koutný,NaN,2024-05-07 18:27:38,2024-05-07 19:45:33,78,Yes,NaN,No,2024-05-07
4,Iveta Poloková,NaN,2024-05-07 18:27:51,2024-05-07 19:57:06,90,Yes,NaN,No,2024-05-07
5,Oskar Koriťák,NaN,2024-05-07 18:30:15,2024-05-07 18:30:28,1,Yes,NaN,No,2024-05-07


### b. seskup údaje podle studentů a spoj vícenásobná připojení

In [38]:
students = reports.groupby([ZOOM_NAME,COL_LESSON]).aggregate({ZOOM_JOIN:'min',ZOOM_LEAVE:'max'}).sort_index()
students

Join Time          Leave Time
Name (Original Name) Lekce                                             
František Matějíček  2024-03-21 2024-03-21 18:18:54 2024-03-21 21:41:39
                     2024-03-25 2024-03-25 18:15:59 2024-03-25 21:34:02
                     2024-03-26 2024-03-26 18:24:16 2024-03-26 21:34:05
                     2024-03-28 2024-03-28 18:26:31 2024-03-28 21:39:31
                     2024-04-01 2024-04-01 18:23:45 2024-04-01 21:30:35
...                                             ...                 ...
Petra Strnadová      2024-04-30 2024-04-30 18:27:53 2024-04-30 21:26:07
                     2024-05-02 2024-05-02 18:27:43 2024-05-02 21:28:05
                     2024-05-06 2024-05-06 18:28:48 2024-05-06 21:20:47
                     2024-05-07 2024-05-07 18:27:05 2024-05-07 19:45:33
Zdeňka Heczková      2024-03-21 2024-03-21 18:18:42 2024-03-21 21:45:21

[163 rows x 2 columns]

### c. spočítej dobu `on-line` v minutách

In [39]:
students[REPORT_DURATION] = (students[ZOOM_LEAVE] - students[ZOOM_JOIN]).dt.seconds //60
students

Join Time          Leave Time  \
Name (Original Name) Lekce                                                
František Matějíček  2024-03-21 2024-03-21 18:18:54 2024-03-21 21:41:39   
                     2024-03-25 2024-03-25 18:15:59 2024-03-25 21:34:02   
                     2024-03-26 2024-03-26 18:24:16 2024-03-26 21:34:05   
                     2024-03-28 2024-03-28 18:26:31 2024-03-28 21:39:31   
                     2024-04-01 2024-04-01 18:23:45 2024-04-01 21:30:35   
...                                             ...                 ...   
Petra Strnadová      2024-04-30 2024-04-30 18:27:53 2024-04-30 21:26:07   
                     2024-05-02 2024-05-02 18:27:43 2024-05-02 21:28:05   
                     2024-05-06 2024-05-06 18:28:48 2024-05-06 21:20:47   
                     2024-05-07 2024-05-07 18:27:05 2024-05-07 19:45:33   
Zdeňka Heczková      2024-03-21 2024-03-21 18:18:42 2024-03-21 21:45:21   

                                 Doba on-line  
Name (Original Name) Lekce                     
František Matějíček  2024-03-21           202  
                     2024-03-25           198  
                     2024-03-26           189  
                     2024-03-28           193  
                     2024-04-01           186  
...                                       ...  
Petra Strnadová      2024-04-30           178  
                     2024-05-02           180  
                     2024-05-06           171  
                     2024-05-07            78  
Zdeňka Heczková      2024-03-21           206  

[163 rows x 3 columns]

### d. vytvoř cílovou složku

In [27]:
''' slozka se zdrojovymi Zoom reporty '''
#DESTINATION_FOLDER = os.path.join(PROJECT_FOLDER,'data','output','reporty','PCS001')

if not os.path.exists(DESTINATION_FOLDER):
    os.makedirs(DESTINATION_FOLDER)

### e. vytvoř individální reporty

In [28]:
for student in reports[ZOOM_NAME].unique():
    print("Pracuji na reportu pro:",student)
    safename = to_ascii(student).replace(' ','')
    file = os.path.join(DESTINATION_FOLDER,f"{safename}.csv")
    print("Pisu do souboru:",file)
    report = students.loc[student]
    print(report)
    report.to_csv(file)

Pracuji na reportu pro: Zdeňka Heczková
Pisu do souboru: c:\Users\Dorota\Desktop\datová analytika\working\data\output\reporty\PCS001\ZdenkaHeczkova.csv
                     Join Time          Leave Time  Doba on-line
Lekce                                                           
2024-03-21 2024-03-21 18:18:42 2024-03-21 21:45:21           206
Pracuji na reportu pro: František Matějíček
Pisu do souboru: c:\Users\Dorota\Desktop\datová analytika\working\data\output\reporty\PCS001\FrantisekMatejicek.csv
                     Join Time          Leave Time  Doba on-line
Lekce                                                           
2024-03-21 2024-03-21 18:18:54 2024-03-21 21:41:39           202
2024-03-25 2024-03-25 18:15:59 2024-03-25 21:34:02           198
2024-03-26 2024-03-26 18:24:16 2024-03-26 21:34:05           189
2024-03-28 2024-03-28 18:26:31 2024-03-28 21:39:31           193
2024-04-01 2024-04-01 18:23:45 2024-04-01 21:30:35           186
2024-04-02 2024-04-02 18:26:29 2024-04